<a href="https://colab.research.google.com/github/iamshouryam/Personal-finance/blob/main/Splitwise_pull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Splitwise Data Extractor for Google Colab - Clean Version

# Install required packages
!pip install requests pandas openpyxl -q

import requests
import pandas as pd
from datetime import datetime, timedelta
import re
from typing import List, Dict, Optional
import time
from google.colab import files

# Configuration
API_KEY = "deEAACvOtNS7Bs89P8Sbs0qx6d7Z0ueYsASz6Yyw"  # Replace with your actual API key
DAYS_BACK = 5  # How many days back to fetch expenses (set to None for all)
OUTPUT_FILENAME = "splitwise_export.xlsx"

# Item categorization mapping
ITEM_CATEGORIES = {
    "tomato": ("Groceries", "Vegetables"), "onion": ("Groceries", "Vegetables"), "eggs": ("Groceries", "Eggs"),
    "dahi": ("Groceries", "Curd"), "paneer": ("Groceries", "Paneer"), "milk": ("Groceries", "Milk"),
    "bhujiya": ("Groceries", "Junk Snacks"), "bread": ("Groceries", "Bread"), "ketchup": ("Groceries", "Sauce"),
    "wrapping paper": ("Stationary", "Stationary"), "aata": ("Groceries", "Rice/Aata/Batter"),
    "tape": ("Stationary", "Stationary"), "chicken": ("Groceries", "Chicken"), "bhindi": ("Groceries", "Vegetables"),
    "chickoo": ("Groceries", "Fruits"), "apple": ("Groceries", "Fruits"), "podi idli powder": ("Gift", "Gift"),
    "chips": ("Groceries", "Junk Snacks"), "maggi": ("Groceries", "Junk Snacks"),
    "burger patty": ("Groceries", "Junk Snacks"), "cream cheese": ("Groceries", "Cheese"),
    "chilli": ("Groceries", "Herbs/lemon/ginger"), "ginger": ("Groceries", "Herbs/lemon/ginger"),
    "banana": ("Groceries", "Fruits"), "cucumber": ("Groceries", "Vegetables"), "guava": ("Groceries", "Fruits"),
    "epigemia protein yogurt": ("Shourya Protein snacks", "Shourya Protein snacks"),
    "protein bar": ("Shourya Protein snacks", "Shourya Protein snacks"), "mouse": ("Miscellaneous", "Electronics"),
    "broccoli": ("Groceries", "Vegetables"), "pasta": ("Groceries", "Noodles/Speghetti"),
    "capsicum": ("Groceries", "Vegetables"), "beans": ("Groceries", "Vegetables"),
    "carrot": ("Groceries", "Vegetables"), "cling wrap": ("Kitchen Tissues/Foils", "Kitchen Tissues/Foils"),
    "lemon": ("Groceries", "Herbs/lemon/ginger"), "aluminium foil": ("Kitchen Tissues/Foils", "Kitchen Tissues/Foils"),
    "condensed milk": ("Groceries", "Special dish condiment"), "cream": ("Groceries", "Special dish condiment"),
    "rakhi": ("Gifting", "Gifting"), "potato": ("Groceries", "Vegetables"), "cheese": ("Groceries", "Cheese"),
    "dhaniya": ("Groceries", "Herbs/lemon/ginger"), "curry leaves": ("Groceries", "Herbs/lemon/ginger"),
    "canola oil": ("Groceries", "Oil/Butter"), "white pepper": ("Groceries", "Spices"),
    "spring onion": ("Groceries", "Herbs/lemon/ginger"), "oyester sauce": ("Groceries", "Condiments"),
    "korean chilli dip": ("Groceries", "Condiments"), "kombucha": ("Shoura Kombucha", "Drink"),
    "tea": ("Groceries", "Tea"), "wai wai": ("Groceries", "Junk Snacks"), "sunflower oil": ("Groceries", "Oil/Butter"),
    "gouda cheese": ("Groceries", "Cheese"), "nescafe gold coffee": ("Shourya Coffee", "Coffee"),
    "sugar": ("Groceries", "Salt/Sugar"), "tumbler": ("Miscellaneous", "Tumbler"),
    "mushroom": ("Groceries", "Vegetables"), "butter": ("Groceries", "Oil/Butter"),
    "chocolate": ("Groceries", "Junk Snacks"), "whey protein": ("Shourya Protein", "Shourya Protein"),
    "garbage bag": ("Kitchen supplies", "Kitchen cleaning supplies"), "harpic": ("Kitchen supplies", "Kitchen cleaning supplies"),
    "black pepper": ("Groceries", "Spices"), "grapes": ("Groceries", "Fruits"), "vinegar": ("Groceries", "Condiments"),
    "sponge scrup": ("Kitchen supplies", "Kitchen cleaning supplies"), "steel scrupper": ("Kitchen supplies", "Kitchen cleaning supplies"),
    "rice": ("Groceries", "Rice/Aata/Batter"), "mint leaves": ("Groceries", "Herbs/lemon/ginger"),
    "husk": ("Shourya Protein snacks", "Shourya Protein snacks"), "protein milkshake": ("Shourya Protein snacks", "Shourya Protein snacks"),
    "penuts": ("Groceries", "Nuts"), "orange": ("Groceries", "Fruits"), "zucchini": ("Groceries", "Vegetables"),
    "coconut": ("Groceries", "Nuts"), "dosa batter": ("Groceries", "Rice/Aata/Batter"),
    "storage bag": ("Miscellaneous", "Storage"), "salt": ("Groceries", "Salt/Sugar"),
    "greek yogurt": ("Groceries", "Curd"), "corn flour": ("Groceries", "Rice/Aata/Batter"),
    "prawns": ("Groceries", "Prawns"), "corn": ("Groceries", "Vegetables"), "avocado": ("Groceries", "Vegetables"),
    "sour cream": ("Groceries", "Cheese"), "coffee": ("Shourya Coffee", "Coffee"),
    "mushroom coffee": ("Shourya Coffee", "Coffee"), "raddish": ("Groceries", "Vegetables"),
    "pear": ("Groceries", "Fruits"), "spinach": ("Groceries", "Vegetables"), "coke": ("Groceries", "Drink"),
    "pumpkin": ("Groceries", "Vegetables"), "amchur powder": ("Groceries", "Spices"),
    "peas": ("Groceries", "Vegetables"), "lotion": ("Toiletries", "Toiletries"),
    "tooth brush": ("Toiletries", "Toiletries"), "dettol": ("Toiletries", "Toiletries"),
    "javetri": ("Groceries", "Spices"), "biryani masala": ("Groceries", "Spices"),
    "black cardamom": ("Groceries", "Spices"), "clove": ("Groceries", "Spices"), "curd": ("Groceries", "Curd"),
    "kewra": ("Groceries", "Spices"), "urad dal": ("Groceries", "Dal"), "ghee": ("Groceries", "Oil/Butter"),
    "kashmiri mirch": ("Groceries", "Spices"), "custard apple": ("Groceries", "Fruits"),
    "sitaphal": ("Groceries", "Fruits"), "robusta banana": ("Groceries", "Fruits"),
    "pachha balehannu": ("Groceries", "Fruits"), "double masala instant noodles": ("Groceries", "Junk Snacks"),
    "akshayakalpa organic probiotic": ("Groceries", "Curd")
}

class SplitwiseExtractor:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://secure.splitwise.com/api/v3.0"
        self.headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

    def test_connection(self) -> bool:
        try:
            response = requests.get(f"{self.base_url}/get_current_user", headers=self.headers)
            if response.status_code == 200:
                user_data = response.json()
                print(f"Connected as: {user_data['user']['first_name']} {user_data['user']['last_name']}")
                return True
            else:
                print(f"Connection failed: {response.status_code}")
                return False
        except Exception as e:
            print(f"Connection error: {str(e)}")
            return False

    def get_all_expenses(self, date_after: Optional[str] = None) -> List[Dict]:
        all_expenses = []
        offset = 0

        while True:
            params = {'limit': 50, 'offset': offset}
            if date_after:
                params['dated_after'] = date_after

            try:
                response = requests.get(f"{self.base_url}/get_expenses", headers=self.headers, params=params)
                if response.status_code != 200:
                    break

                expenses = response.json().get('expenses', [])
                if not expenses:
                    break

                all_expenses.extend(expenses)
                print(f"Fetched {len(all_expenses)} expenses")

                if len(expenses) < 50:
                    break

                offset += 50
                time.sleep(0.5)

            except Exception as e:
                print(f"Error: {str(e)}")
                break

        return all_expenses

    def get_item_category(self, item_name: str) -> tuple:
        clean_name = item_name.lower().strip()
        clean_name = re.sub(r'^\d+\s*x\s*', '', clean_name)
        clean_name = re.sub(r'\[combo\]\s*', '', clean_name)
        clean_name = re.sub(r'\(.*?\)', '', clean_name)
        clean_name = clean_name.strip()

        if clean_name in ITEM_CATEGORIES:
            return ITEM_CATEGORIES[clean_name]

        for key, category in ITEM_CATEGORIES.items():
            if key in clean_name or clean_name in key:
                return category

        return ("Miscellaneous", "Uncategorized")

    def parse_itemized_details(self, details: str, expense_description: str = "", expense_date: str = "") -> List[Dict]:
        if not details:
            return []

        items = []
        merchant_name = "Unknown"

        # Extract merchant
        for pattern in [r'Merchant name:\s*(.+)', r'Store:\s*(.+)', r'From:\s*(.+)']:
            match = re.search(pattern, details, re.IGNORECASE)
            if match:
                merchant_name = match.group(1).strip()
                break

        if merchant_name == "Unknown" and expense_description:
            merchant_name = expense_description.split(' - ')[0] if ' - ' in expense_description else expense_description[:30]

        # Parse item patterns
        patterns = [
            r'(\d+)\s*x\s*(.+?)\s*[-–—]\s*([\d,]+\.?\d*)\s*\(.*?\)',
            r'(\d+)\s*x\s*(.+?)\s*[-–—]\s*([\d,]+\.?\d*)',
            r'^(.+?)\s*[-–—]\s*\$?([\d,]+\.?\d*)\s*\([^)]+\)$',
            r'^(.+?)\s*[-–—]\s*\$?([\d,]+\.?\d*)$',
            r'^(.+?):\s*\$?([\d,]+\.?\d*)$',
            r'^\$?([\d,]+\.?\d*)\s+(.+)$',
            r'^(.+?)\s*\(\$?([\d,]+\.?\d*)\)$'
        ]

        for line in details.strip().split('\n'):
            line = line.strip()
            if not line or any(x in line for x in ['Merchant name:', 'Purchase', 'Service charge:']):
                continue

            quantity, item_name, amount = 1, "", None

            for pattern in patterns:
                match = re.match(pattern, line, re.IGNORECASE)
                if match:
                    groups = match.groups()

                    if len(groups) == 3:
                        try:
                            quantity = int(groups[0])
                            item_name = groups[1].strip()
                            amount = float(groups[2].replace(',', '').replace('$', ''))
                            break
                        except ValueError:
                            continue

                    elif len(groups) == 2:
                        try:
                            try:
                                amount = float(groups[0].replace(',', '').replace('$', ''))
                                item_name = groups[1].strip()
                            except ValueError:
                                amount = float(groups[1].replace(',', '').replace('$', ''))
                                item_name = groups[0].strip()
                            break
                        except ValueError:
                            continue

            if not item_name:
                item_name = line

            category, subcategory = self.get_item_category(item_name)
            clean_item_name = re.sub(r'\[combo\]\s*', '', item_name, flags=re.IGNORECASE).strip()

            items.append({
                'item_name': clean_item_name,
                'category': category,
                'subcategory': subcategory,
                'quantity': quantity,
                'unit_price': amount / quantity if amount and quantity > 0 else amount,
                'total_price': amount,
                'merchant': merchant_name,
                'expense_date': expense_date
            })

        return items

    def extract_to_dataframe(self, expenses: List[Dict]) -> pd.DataFrame:
        itemized_data = []

        for expense in expenses:
            details = expense.get('details', '')
            expense_desc = expense.get('description', '')
            expense_date = expense.get('date', '')
            itemized_items = self.parse_itemized_details(details, expense_desc, expense_date)

            for item in itemized_items:
                itemized_data.append({
                    'item_name': item.get('item_name', ''),
                    'category': item.get('category', ''),
                    'subcategory': item.get('subcategory', ''),
                    'quantity': item.get('quantity', 1),
                    'unit_price': item.get('unit_price'),
                    'total_price': item.get('total_price'),
                    'merchant': item.get('merchant', ''),
                    'expense_date': item.get('expense_date', ''),
                    'expense_id': expense.get('id')
                })

        return pd.DataFrame(itemized_data)

    def export_to_excel(self, df: pd.DataFrame, filename: str) -> str:
        df.to_excel(filename, index=False)
        return filename

def main():
    if API_KEY == "YOUR_API_KEY_HERE":
        print("ERROR: Please replace API_KEY with your actual Splitwise API key!")
        return

    extractor = SplitwiseExtractor(API_KEY)

    if not extractor.test_connection():
        print("Connection failed. Check your API key.")
        return

    date_after = None
    if DAYS_BACK:
        date_after = (datetime.now() - timedelta(days=DAYS_BACK)).strftime("%Y-%m-%dT00:00:00Z")

    expenses = extractor.get_all_expenses(date_after=date_after)
    if not expenses:
        print("No expenses found!")
        return

    df = extractor.extract_to_dataframe(expenses)
    filename = extractor.export_to_excel(df, OUTPUT_FILENAME)

    print(f"\nExport completed: {filename}")
    print(f"Extracted {len(df)} itemized entries")

    files.download(filename)

# Run the script
main()

Connected as: Shourya Mehta
Fetched 12 expenses

Export completed: splitwise_export.xlsx
Extracted 8 itemized entries


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>